# HMM Align using trained models

In [ ]:
%run ../prongen/hmm_pron.py --in-jupyter
%run ../acmodel/plot.py
%matplotlib ipympl
%run ../acmodel/matrix.py
%run ../acmodel/praat_ifc.py
%run ../acmodel/gmm_model_data.py
%run ../acmodel/hmm_acmodel.py

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
import pandas as pd
df = pd.read_csv("maxi.tsv", sep="\t", keep_default_na=False)  # read mini.tsv or maxi.tsv

In [ ]:
hmms = []
#for wav, sentence in list(zip(df.wav.values, df.sentence.values))[:50]:
for wav, sentence in list(zip(df.wav.values, df.sentence.values)):
    #print(sentence)
    hmm = HMM(sentence, wav=wav)
    #hmm.change_sil_loops()
    hmms.append(hmm)
    #print(hmm)

In [ ]:
hmm = hmms[4] # Občasně konzumuje i větší hmyz a výjimečně i ryby.
#hmm = hmms[9] # To mělo za následek některé fatální důsledky.
#hmm = hmms[12] # Bílkoviny většinou vykazují v oblasti spoje prvky symetrie.
#hmm = hmms[19] # Rovina řezu je horizontální.
#hmm = hmms[22] # Jeho zájmy jsou skauting a florbal.
#hmm = hmms[23] # Nemáme právo stavět se v této roztržce na něčí stranu.
#hmm = hmms[24] # Model financování se odráží i v činnosti operních divadel.

# mini:
# hmm = hmms[4] # Co to znamená?
#hmm = hmms[0] # Zdrželi se?
#hmm = hmms[1] # Ne, klesáme!
#hmm = hmms[2] # Ne, neudělala.
#hmm = hmms[3] # To není fér!
#hmm = hmms[5] # Ona to ví!
#hmm = hmms[6] # Ale, ale ale.

print(hmm.orto)

## Compute alignment

In [ ]:
alpha_m, alpha_exp = viterbi_align(hmm, b_means)
alp = torch.cat(alpha_m.val)
intervals, bap = backward_alignment_pass_intervals(hmm, alp) # intervals will be sent to praat
#intervals

In [ ]:
i = bap.max(1).indices
s = "".join([hmm.b[ii] for ii in i])
s

In [ ]:
troubling_alignmet(s)

In [ ]:
troubling_alignmet(s)

In [ ]:
plot_matrix(torch.cat([bap, alp], dim=1))

In [ ]:
tft = textgrid_file_text({"segmenty": intervals})
!cp {hmm.wav} test.wav

In [ ]:
%%writetemplate test.TextGrid
{tft}

## Alignment is now ready for inspection in praat
Open files ```test.wav``` and ```test.TextGrid```

## Aditional analysis

In [ ]:
b = m(compute_hmm_b(hmm, b_means).clamp(min=0.00001))

plot_fun(b.val[:,[3]].log())

In [ ]:
plot_wavfile(hmm.wav)

In [ ]:

plot_fun(hmm.mfcc[:,0])

In [ ]:
def looks_like_speech(mfcc):
    """
    Fuzzy detector of speech. Intended as a tweek of b() for silence.
    """
    eng = mfcc[:,0]
    maximum = eng.max()
    minimum = eng.min()
    mid = (maximum+minimum)/2
    above = eng>mid
    return above
    
    
    
    
    

In [ ]:
plot_fun(looks_like_speech(hmm.mfcc))

## Cell Magic, %%writetemplate, %%praat


sendpraat source and compilation and usage: https://www.fon.hum.uva.nl/praat/sendpraat.html

Praat history and scripting: https://www.fon.hum.uva.nl/praat/manual/History_mechanism.html

In [ ]:
@register_line_cell_magic
def praat(line, cell):
    with open("/Users/vhanzl/tmp/tmp-script.praat", 'w') as f:
        f.write(cell.format(**globals()))
    !~/bin/sendpraat 100 praat "runScript: \"/Users/vhanzl/tmp/tmp-script.praat\""

In [ ]:
@register_line_cell_magic
def praat(line, cell):
    script = cell.format(**globals()).replace('"','\\"')
    !~/bin/sendpraat 600 praat "{script}"

In [ ]:
some_variable = "text_to_be_shown"

In [ ]:
%%writetemplate script_for_praat.txt
printline {some_variable}

In [ ]:
!praat --send script_for_praat.txt

In [ ]:
script = """
printline milujeme Praat
"""
!~/bin/sendpraat 600 praat "{script}"

In [ ]:
!~/bin/sendpraat 100 praat "Read from file... {AUDIODIR / FILE}"
!~/bin/sendpraat 100 praat "View & Edit"

## Compute alignments for all HMMs

In [ ]:
%%time
for idx, hmm in enumerate(hmms):
    if idx%100==0:
        print(idx)
    
    alpha_m, alpha_exp = viterbi_align(hmm, b_means)
    alp = torch.cat(alpha_m.val)
    hmm.intervals, bap = backward_alignment_pass_intervals(hmm, alp)
    hmm.indices = i = bap.max(1).indices
    s = "".join([hmm.b[ii] for ii in i])
    hmm.troubling = troubling_alignmet(s)
    hmm.targets = "".join([hmm.b[ii] for ii in i])

# While also training in parallel:  CPU times: user 45min 57s, sys: 22.6 s, total: 46min 20s  Wall time: 5min 49s

In [ ]:
df['targets'] = [hmm.targets for hmm in hmms]

In [ ]:
df

In [ ]:
df.to_csv("nn_train.tsv", sep="\t", index=False)

In [ ]:
!head nn_train.tsv

In [ ]:
durs = {}
for hmm in hmms:
    #for f, t, ph in hmm.intervals:
    for f, t, ph in hmm.intervals[1:-1]:
        duration = t-f
        if ph not in durs:
            durs[ph] = []
        durs[ph].append(duration)  

In [ ]:
for phone, dur in durs.items():
    print(f"Mean duration of {phone}: {'%0.3f' % tensor(durs[phone]).mean()}   sum: {'%0.3f' % tensor(durs[phone]).sum()}   count: {len(durs[phone])}")

In [ ]:
for phone, dur in durs.items():
    print(f"Mean duration of {phone}: {'%0.3f' % tensor(durs[phone]).mean()}   sum: {'%0.3f' % tensor(durs[phone]).sum()}   count: {len(durs[phone])}")

In [ ]:
len(durs['s'])

In [ ]:
s = ''
t = ''
for hmm in hmms:
    s += hmm.b
    t += hmm.troubling
    
{*t}

In [ ]:
len(t)